In [2]:
import pandas as pd
import numpy as np
import warnings
import os
import sys
import urllib.request
import joblib
import simplejson as json
from time import time
warnings.filterwarnings(action='ignore')

import seaborn as sns
sns.set_palette("pastel")
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline

import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')

In [3]:
#LPD_201902 = pd.read_csv('LOCAL_PEOPLE_DONG_201902.csv',index_col=False)
#dfcolumns = pd.read_csv('LOCAL_PEOPLE_DONG_201903.csv',
#                        nrows = 1)
#LPD_201903 = pd.read_csv('LOCAL_PEOPLE_DONG_201903.csv', header = None, 
#                         skiprows = 1, usecols = list(range(len(dfcolumns.columns))), 
#                         names = dfcolumns.columns)
#LPD_201904 = pd.read_csv('LOCAL_PEOPLE_DONG_201904.csv',index_col=False)
#LPD_201905 = pd.read_csv('LOCAL_PEOPLE_DONG_201905.csv',index_col=False)
#LPD_202002 = pd.read_csv('LOCAL_PEOPLE_DONG_202002.csv',index_col=False)
#LPD_202003 = pd.read_csv('LOCAL_PEOPLE_DONG_202003.csv',index_col=False)
#LPD_202004 = pd.read_csv('LOCAL_PEOPLE_DONG_202004.csv',index_col=False)
#LPD_202005 = pd.read_csv('LOCAL_PEOPLE_DONG_202005.csv',index_col=False)

In [4]:
LPD_202006 = pd.read_csv('LOCAL_PEOPLE_DONG_202006.csv',index_col=False)
LPD_202007 = pd.read_csv('LOCAL_PEOPLE_DONG_202007.csv',index_col=False)

In [5]:
df = pd.concat([LPD_202006, LPD_202007])

In [6]:
df.rename(columns = {'남자0세부터9세생활인구수':'남자_0009세생활인구수'},inplace=True)
df.rename(columns = {'남자70세이상생활인구수':'남자_70세이상생활인구수'},inplace=True)
df.rename(columns = {'여자0세부터9세생활인구수':'여자_0009세생활인구수'},inplace=True)
df.rename(columns = {'여자70세이상생활인구수':'여자_70세이상생활인구수'},inplace=True)

In [7]:
for i in np.arange(1,13):
    temp = list(df.columns[4:])[i][:2]+'_'+list(df.columns[4:])[i][2:4]+list(df.columns[4:])[i][7:]
    name = list(df.columns[4:])[i]
    df.rename(columns={name:temp},inplace=True)

for i in np.arange(15,27):
    temp = list(df.columns[4:])[i][:2]+'_'+list(df.columns[4:])[i][2:4]+list(df.columns[4:])[i][7:]
    name = list(df.columns[4:])[i]
    df.rename(columns={name:temp},inplace=True)

In [8]:
cd = pd.read_csv('행정동코드_매핑정보.csv',encoding='cp949',header=1)
cd['G_DNG_NM'] = cd[['DO_NM','CT_NM','H_DNG_NM']].apply(lambda x: ' '.join(x), axis=1)
GU_DONG = cd[['H_DNG_CD','G_DNG_NM']]
GU_DONG.rename(columns = {'H_DNG_CD':'행정동코드'},inplace=True)

In [9]:
life_pup = pd.merge(df,GU_DONG,on='행정동코드',how='inner')

In [10]:
life_pup = pd.concat([life_pup[life_pup['G_DNG_NM'].str.contains('노원구')], life_pup[life_pup['G_DNG_NM'].str.contains('중구')]])

In [11]:
life_pup.rename(columns = {'기준일ID':'STD_DD'},inplace=True)

In [12]:
life_pup['STD_DD'] = life_pup['STD_DD'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

In [13]:
life_pup = life_pup.reset_index(drop=True)

In [16]:
life_pup = life_pup.drop(columns=['행정동코드','총생활인구수'])

In [23]:
pup = pd.melt(life_pup,id_vars=['STD_DD','시간대구분','G_DNG_NM'],var_name='HUMAN_NM',value_name='FLOW')

In [24]:
pup['SEX_CD'] = np.where(pup['HUMAN_NM'].str.contains('남자'),'M','F')
pup['HDONG_NM'] = pup['G_DNG_NM'].str[6:].apply(lambda x: x.strip())
pup['WEEK_NUM'] = pd.to_datetime(pup['STD_DD']).dt.weekofyear
pup['AGE_CD'] = pup['HUMAN_NM'].str[3:8]
pup['WEEKDAY'] = pd.to_datetime(pup['STD_DD']).dt.weekday

In [27]:
pup = pup.drop(columns='HUMAN_NM')

In [29]:
pup.to_csv('local_people_melt_0607.csv',index=False)

In [22]:
pd.read_csv('local_people_melt.csv')

,STD_DD,시간대구분,행정동코드,G_DNG_NM,HUMAN_NM,FLOW,SEX_CD,HDONG_NM,WEEK_NUM,AGE_CD,WEEKDAY
0,2019-02-01,0,11350625,서울 노원구 중계2.3동,남자_0009세생활인구수,1858.3096,M,중계2.3동,5,0009세,4
1,2019-02-01,1,11350625,서울 노원구 중계2.3동,남자_0009세생활인구수,1785.0549,M,중계2.3동,5,0009세,4
2,2019-02-01,2,11350625,서울 노원구 중계2.3동,남자_0009세생활인구수,1733.5305,M,중계2.3동,5,0009세,4
3,2019-02-01,3,11350625,서울 노원구 중계2.3동,남자_0009세생활인구수,1745.2137,M,중계2.3동,5,0009세,4
4,2019-02-01,4,11350625,서울 노원구 중계2.3동,남자_0009세생활인구수,1712.3663,M,중계2.3동,5,0009세,4
...,...,...,...,...,...,...,...,...,...,...,...
5506363,2020-05-31,19,11140605,서울 중구 을지로동,여자_70세이상생활인구수,301.2843,F,을지로동,22,70세이상,6
5506364,2020-05-31,20,11140605,서울 중구 을지로동,여자_70세이상생활인구수,301.9059,F,을지로동,22,70세이상,6
5506365,2020-05-31,21,11140605,서울 중구 을지로동,여자_70세이상생활인구수,246.0693,F,을지로동,22,70세이상,6
5506366,2020-05-31,22,11140605,서울 중구 을지로동,여자_70세이상생활인구수,251.1501,F,을지로동,22,70세이상,6
